<a href="https://colab.research.google.com/github/angelodilegge2/Code-templates/blob/main/Extract_location_and_names_in_Apple_photos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the TSV file
input_file = 'photo_metadata.csv'  # Adjust file name as needed
df = pd.read_csv(input_file, delimiter='\t')

# Display column names to confirm the file is loaded correctly
print("Columns in the DataFrame:", df.columns)

# Preview the data
print("Loaded data:")
print(df.head())

Columns in the DataFrame: Index(['Date', 'Latitude', 'Longitude', 'Person'], dtype='object')
Loaded data:
                  Date   Latitude  Longitude            Person
0  2014-01-19 14:54:50  41.905937  12.482242   Chiara Garzilli
1  2021-03-06 18:36:01  50.836655   4.348172  Francesco Romano
2  2020-07-11 21:32:06  50.864545   4.346442  Francesco Romano
3  2020-06-26 19:13:48  51.315320   3.123114  Francesco Romano
4  2020-08-27 17:43:02  50.837883   4.368147  Francesco Romano


In [ ]:
# Filter out invalid latitude and longitude values
df = df[(df['Latitude'] != -180.0) & (df['Longitude'] != -180.0)]

# Display filtered data
print("Filtered data (valid coordinates):")
print(df.head())

Filtered data (valid coordinates):
         Date   Latitude  Longitude            Person
0  2014-01-19  41.905937  12.482242   Chiara Garzilli
1  2021-03-06  50.836655   4.348172  Francesco Romano
2  2020-07-11  50.864545   4.346442  Francesco Romano
3  2020-06-26  51.315320   3.123114  Francesco Romano
4  2020-08-27  50.837883   4.368147  Francesco Romano


In [ ]:
# Ensure 'DateTaken' is in datetime format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Remove the hour from 'DateTaken' column, keeping only the date
df['Date'] = df['Date'].dt.date

# Display the updated data to confirm the transformation
print("Date column after removing the hour:")
print(df[['Date']].head())

Date column after removing the hour:
         Date
0  2014-01-19
1  2021-03-06
2  2020-07-11
3  2020-06-26
4  2020-08-27


In [ ]:
# Count the number of rows left in the DataFrame
print(f"Number of entries left: {len(df)}")

Number of entries left: 17834


In [ ]:
from geopy.geocoders import Nominatim
from unidecode import unidecode

# Initialize geolocator
geolocator = Nominatim(user_agent="geo_translator")

# Reverse Geocoding Function with English/Latin alphabet conversion
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if location and location.raw.get('address'):
            address = location.raw['address']
            city = address.get('city') or address.get('town') or address.get('village') or address.get('hamlet')
            country = address.get('country')

            # Transliterate to Latin alphabet if necessary
            city = unidecode(city) if city else None
            country = unidecode(country) if country else None

            return f"{city}, {country}" if city else country
        return "Unknown Location"
    except Exception as e:
        print(f"Error with coordinates ({lat}, {lon}): {e}")
        return "Unknown Location"

In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
from tqdm import tqdm

# Enable tqdm for pandas
tqdm.pandas()

In [ ]:
# Apply reverse geocoding with a progress bar
print("Starting reverse geocoding...")
df['Address'] = df.progress_apply(lambda row: reverse_geocode(row['Latitude'], row['Longitude']), axis=1)
print("Reverse geocoding complete.")

Starting reverse geocoding...


100%|██████████| 17834/17834 [2:29:46<00:00,  1.98it/s]

Reverse geocoding complete.


In [ ]:
# Save the results to a new CSV file
output_file = 'photo_metadata_with_addresses.csv'
df.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

Results saved to photo_metadata_with_addresses.csv


In [ ]:
import pandas as pd

# Load the photo metadata
photo_file = 'photo_metadata_with_addresses.csv'  # Replace with your file path
photo_df = pd.read_csv(photo_file)

# Ensure the Date column is in datetime format
photo_df['Date'] = pd.to_datetime(photo_df['Date'], errors='coerce')

# Extract year and month for grouping
photo_df['Year'] = photo_df['Date'].dt.year
photo_df['Month'] = photo_df['Date'].dt.month_name()  # Get the full name of the month

# Ensure Address and Person columns are strings, replacing NaN with an empty string
photo_df['Address'] = photo_df['Address'].fillna('').astype(str)
photo_df['Person'] = photo_df['Person'].fillna('').astype(str)

# Group by year and month, and aggregate unique locations and people
photo_summary = (
    photo_df.groupby(['Year', 'Month'])
    .agg({
        'Address': lambda x: ', '.join(sorted(set(x))),
        'Person': lambda x: ', '.join(sorted(set(x)))
    })
    .reset_index()
    .rename(columns={'Address': 'Locations', 'Person': 'People'})
)

# Define the full range of years for analysis data
analysis_data = [
    {"Year": year}  # Only the Year column is included
    for year in range(1984, 2025)  # Update to match your desired year range
]

# Convert analysis data into a DataFrame
analysis_df = pd.DataFrame(analysis_data)

# Merge the photo summary with the analysis data on Year
merged_df = pd.merge(analysis_df, photo_summary, on='Year', how='left')

# Save the combined data to a CSV file
output_file = 'combined_summary_analysis.csv'
merged_df.to_csv(output_file, index=False)

# Display the merged table
print("Combined analysis and photo summary:")
print(merged_df)

Combined analysis and photo summary:
     Year      Month                                          Locations  \
0    1984        NaN                                                NaN   
1    1985        NaN                                                NaN   
2    1986        NaN                                                NaN   
3    1987        NaN                                                NaN   
4    1988        NaN                                                NaN   
..    ...        ...                                                ...   
193  2024      March  Bruxelles - Brussel, Belgie / Belgique / Belgi...   
194  2024        May  Aachen, Deutschland, Belgie / Belgique / Belgi...   
195  2024   November  Antwerpen, Belgie / Belgique / Belgien, Bruxel...   
196  2024    October  Bruxelles - Brussel, Belgie / Belgique / Belgi...   
197  2024  September  Durbuy, Belgie / Belgique / Belgien, Terracina...   

                                                People  
0    

In [ ]:
import pandas as pd

# Load the photo metadata
photo_file = 'photo_metadata_with_addresses.csv'  # Replace with your file path
photo_df = pd.read_csv(photo_file)

# Ensure the Date column is in datetime format
photo_df['Date'] = pd.to_datetime(photo_df['Date'], errors='coerce')

# Extract year and month for grouping
photo_df['Year'] = photo_df['Date'].dt.year
photo_df['Month'] = photo_df['Date'].dt.month_name()  # Get the full name of the month

# Ensure Address and Person columns are strings, replacing NaN with empty strings
photo_df['Address'] = photo_df['Address'].fillna('').astype(str)
photo_df['Person'] = photo_df['Person'].fillna('').astype(str)

# Group by year and month, and aggregate unique locations and people
photo_summary = (
    photo_df.groupby(['Year', 'Month'])
    .agg({
        'Address': lambda x: ', '.join(sorted(set(x))),
        'Person': lambda x: ', '.join(sorted(set(x)))
    })
    .reset_index()
    .rename(columns={'Address': 'Locations', 'Person': 'People'})
)

# Full analysis data
analysis_data = [
    {"Year": 1984, "Age": 0, "Location": "Priverno (Latina)", "Main Occupation": "Early Childhood",
     "Description": "Born on 9 July 1984 in Priverno, Italy.", "% Studying": "0%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1985, "Age": 1, "Location": "Priverno (Latina)", "Main Occupation": "Early Childhood",
     "Description": "Early childhood in Priverno.", "% Studying": "0%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1986, "Age": 2, "Location": "Priverno (Latina)", "Main Occupation": "Early Childhood",
     "Description": "Early childhood in Priverno.", "% Studying": "0%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1987, "Age": 3, "Location": "Priverno (Latina)", "Main Occupation": "Early Childhood",
     "Description": "Began Scuola Materna (preschool).", "% Studying": "0%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1988, "Age": 4, "Location": "Priverno (Latina)", "Main Occupation": "Early Childhood",
     "Description": "Continued Scuola Materna.", "% Studying": "0%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1989, "Age": 5, "Location": "Priverno (Latina)", "Main Occupation": "Early Childhood",
     "Description": "Completed Scuola Materna and prepared for primary school.", "% Studying": "0%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1990, "Age": 6, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Began Scuola Elementare (primary school).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1991, "Age": 7, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Continued primary school (Year 2).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1992, "Age": 8, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Continued primary school (Year 3).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1993, "Age": 9, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Continued primary school (Year 4).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1994, "Age": 10, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Completed primary school (Year 5).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1995, "Age": 11, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Began Scuola Media (middle school).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1996, "Age": 12, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Continued middle school (Year 2).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1997, "Age": 13, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Completed middle school (Year 3).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 1998, "Age": 14, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Began Liceo Scientifico (high school).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
{"Year": 1999, "Age": 15, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Continued high school (Year 2).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 2000, "Age": 16, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Continued high school (Year 3).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 2001, "Age": 17, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Continued high school (Year 4).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 2002, "Age": 18, "Location": "Priverno (Latina)", "Main Occupation": "Study",
     "Description": "Final year of high school; graduated with 84/100.", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 2003, "Age": 19, "Location": "Rome", "Main Occupation": "Study",
     "Description": "Began a bachelor’s degree in Political Science and International Relations.", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 2004, "Age": 20, "Location": "Rome", "Main Occupation": "Study",
     "Description": "Continued bachelor’s degree studies (Year 2).", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 2005, "Age": 21, "Location": "Rome", "Main Occupation": "Study",
     "Description": "Final year of bachelor’s degree; focused on thesis work.", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 2006, "Age": 22, "Location": "Rome and Athens", "Main Occupation": "Study and Work",
     "Description": "Graduated with 102/110. Worked at the EPLO in Athens (4.5 months).", "% Studying": "~63%", "% Working": "~37%", "Income (EUR)": 5000},
    {"Year": 2007, "Age": 23, "Location": "Rome", "Main Occupation": "Study and Work",
     "Description": "Began a master’s degree in Business Administration in January. Started Servizio Civile in September.", "% Studying": "~50%", "% Working": "~50%", "Income (EUR)": None},
    {"Year": 2008, "Age": 24, "Location": "Rome and Maastricht", "Main Occupation": "Study and Work",
     "Description": "Completed Servizio Civile in June. Participated in Erasmus in Maastricht from September to December.", "% Studying": "~50%", "% Working": "~50%", "Income (EUR)": 3905},
    {"Year": 2009, "Age": 25, "Location": "Rome and Hong Kong", "Main Occupation": "Study and Work",
     "Description": "Continued master’s degree; worked at the Consulate General of Italy (4 months).", "% Studying": "~66.7%", "% Working": "~33.3%", "Income (EUR)": None},
    {"Year": 2010, "Age": 26, "Location": "Rome", "Main Occupation": "Study and Work",
     "Description": "Graduated with 110/110. Worked at PwC in an apprentice position (7 months).", "% Studying": "~36.4%", "% Working": "~63.6%", "Income (EUR)": 10097},
    {"Year": 2011, "Age": 27, "Location": "Rome and London", "Main Occupation": "Study and Work",
     "Description": "Left PwC in May. Began a Master of Public Administration (MPA) at LSE in September.", "% Studying": "~66.7%", "% Working": "~33.3%", "Income (EUR)": None},
    {"Year": 2012, "Age": 28, "Location": "London", "Main Occupation": "Study",
     "Description": "Continued MPA studies at LSE.", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 2013, "Age": 29, "Location": "London and Rome", "Main Occupation": "Study and Work",
     "Description": "Completed MPA with Merit. Worked at VVA Europe Limited (8 months).", "% Studying": "~33.3%", "% Working": "~66.7%", "Income (EUR)": None},
    {"Year": 2014, "Age": 30, "Location": "Milan", "Main Occupation": "Work",
     "Description": "Began working for PwC Advisory in Milan.", "% Studying": "0%", "% Working": "100%", "Income (EUR)": 18671},
    {"Year": 2015, "Age": 31, "Location": "Milan", "Main Occupation": "Work",
     "Description": "Continued working at PwC Advisory; transitioned to EY Advisory in October.", "% Studying": "0%", "% Working": "100%", "Income (EUR)": 31786},
    {"Year": 2016, "Age": 32, "Location": "Milan", "Main Occupation": "Work",
     "Description": "Continued working at EY Advisory.", "% Studying": "0%", "% Working": "100%", "Income (EUR)": 36829},
    {"Year": 2017, "Age": 33, "Location": "Milan", "Main Occupation": "Study and Work",
     "Description": "Prepared for the diplomatic concourse; left EY in January (worked 1 month).", "% Studying": "~91.7%", "% Working": "~8.3%", "Income (EUR)": 4990},
    {"Year": 2018, "Age": 34, "Location": "Milan", "Main Occupation": "Study",
     "Description": "Full-time preparation for the diplomatic concourse.", "% Studying": "100%", "% Working": "0%", "Income (EUR)": None},
    {"Year": 2019, "Age": 35, "Location": "Milan and Brussels", "Main Occupation": "Study and Work",
     "Description": "Studied for the diplomatic concourse until September; began work at VVA Europe.", "% Studying": "~75%", "% Working": "~25%", "Income (EUR)": 7392},
    {"Year": 2020, "Age": 36, "Location": "Brussels", "Main Occupation": "Work",
     "Description": "Continued working at VVA Europe.", "% Studying": "0%", "% Working": "100%", "Income (EUR)": 37390},
    {"Year": 2021, "Age": 37, "Location": "Brussels", "Main Occupation": "Work",
     "Description": "Continued working at VVA Europe.", "% Studying": "0%", "% Working": "100%", "Income (EUR)": 38921},
    {"Year": 2022, "Age": 38, "Location": "Brussels", "Main Occupation": "Work",
     "Description": "Continued working at VVA Europe; transitioned to NTT DATA in September.", "% Studying": "0%", "% Working": "100%", "Income (EUR)": 45416},
    {"Year": 2023, "Age": 39, "Location": "Brussels", "Main Occupation": "Work",
     "Description": "Worked at NTT DATA; experienced a career interruption for 87 days.", "% Studying": "0%", "% Working": "100%", "Income (EUR)": 60046},
    {"Year": 2024, "Age": 40, "Location": "Brussels", "Main Occupation": "Study and Work",
     "Description": "Left NTT DATA in February. Prepared for a PhD in Economics. Started freelance work in September.",
     "% Studying": "~50%", "% Working": "~50%", "Income (EUR)": None},
]

# Convert analysis data into a DataFrame
analysis_df = pd.DataFrame(analysis_data)

# Merge the photo summary with the analysis data on Year
merged_df = pd.merge(analysis_df, photo_summary, on='Year', how='left')

# Save the combined data to a CSV file
output_file = 'combined_summary_analysis.csv'
merged_df.to_csv(output_file, index=False)

# Display the merged table
print("Combined analysis and photo summary:")
print(merged_df)

Combined analysis and photo summary:
     Year  Age           Location  Main Occupation  \
0    1984    0  Priverno (Latina)  Early Childhood   
1    1985    1  Priverno (Latina)  Early Childhood   
2    1986    2  Priverno (Latina)  Early Childhood   
3    1987    3  Priverno (Latina)  Early Childhood   
4    1988    4  Priverno (Latina)  Early Childhood   
..    ...  ...                ...              ...   
193  2024   40           Brussels   Study and Work   
194  2024   40           Brussels   Study and Work   
195  2024   40           Brussels   Study and Work   
196  2024   40           Brussels   Study and Work   
197  2024   40           Brussels   Study and Work   

                                           Description % Studying % Working  \
0              Born on 9 July 1984 in Priverno, Italy.         0%        0%   
1                         Early childhood in Priverno.         0%        0%   
2                         Early childhood in Priverno.         0%        0%  

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm import tqdm

# Load your dataset
file_path = 'photo_metadata_with_addresses.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Initialize the geolocator
geolocator = Nominatim(user_agent="geo_locator")

# Function to get the country of a location
def get_country(location):
    try:
        geo_location = geolocator.geocode(location)
        if geo_location and geo_location.raw.get('address'):
            return geo_location.raw['address'].get('country', 'Unknown Country')
    except Exception as e:
        print(f"Error fetching country for {location}: {e}")
    return "Unknown Country"

# Get unique locations
unique_locations = df['Address'].dropna().unique()

# Add progress bar
tqdm.pandas()

# Create a DataFrame with unique locations and their countries
locations_with_countries = pd.DataFrame({
    'Location': unique_locations,
    'Country': [get_country(location) for location in tqdm(unique_locations, desc="Processing Locations")]
})

# Save the results to a CSV file
output_file_path = 'locations_with_countries.csv'
locations_with_countries.to_csv(output_file_path, index=False)

print(f"CSV file created at: {output_file_path}")

Processing Locations: 100%|██████████| 313/313 [02:39<00:00,  1.96it/s]

CSV file created at: locations_with_countries.csv


In [ ]:
import pandas as pd

# Load the dataset
file_path = 'locations_with_countries.csv'  # Use your actual file path
data = pd.read_csv(file_path)

# Extract city and country from the Location column
data[['City', 'ExtractedCountry']] = data['Location'].str.split(',', n=1, expand=True)

# Clean up extracted data
data['City'] = data['City'].str.strip()  # Remove leading/trailing whitespace
data['ExtractedCountry'] = data['ExtractedCountry'].str.strip()  # Remove whitespace

# Replace "Unknown" in the Country column with the extracted country
data['Country'] = data['Country'].where(data['Country'] != 'Unknown', data['ExtractedCountry'])

# Drop the temporary 'ExtractedCountry' column
data = data.drop(columns=['ExtractedCountry'])

# Save the cleaned data to a new CSV file
output_file_path = 'processed_locations_with_countries.csv'
data.to_csv(output_file_path, index=False)

print(f"Processed CSV file saved at: {output_file_path}")

Processed CSV file saved at: processed_locations_with_countries.csv


In [ ]:
import pandas as pd

# Load your dataset
file_path = 'processed_locations_with_countries.csv'  # Replace with the actual file path
df = pd.read_csv(file_path)

# Group by country and count the unique cities visited
country_exploration = df.groupby('Country')['Location'].nunique().reset_index()

# Rename columns for clarity
country_exploration.columns = ['Country', 'UniqueCitiesVisited']

# Add an exploration rank (higher rank for more cities visited)
country_exploration['ExplorationRank'] = country_exploration['UniqueCitiesVisited'].rank(method='dense', ascending=False).astype(int)

# Save the results to a new CSV file
output_file_path = 'country_exploration_index.csv'
country_exploration.to_csv(output_file_path, index=False)

print(f"Country Exploration Index CSV created at: {output_file_path}")

Country Exploration Index CSV created at: country_exploration_index.csv


In [ ]:
import pandas as pd

# Load the dataset
file_path = "photo_metadata_with_addresses.csv"  # Replace with the actual file path
data = pd.read_csv(file_path)

# Extract the "Address" column and remove duplicates
address_column = data[['Address']].drop_duplicates()

# Save the deduplicated column to a new CSV file
output_path = "extracted_addresses.csv"  # Replace with the desired output file path
address_column.to_csv(output_path, index=False)

print(f"The deduplicated 'Address' column has been successfully extracted and saved to {output_path}")

The deduplicated 'Address' column has been successfully extracted and saved to extracted_addresses.csv
